# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra
using ASEconvert

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al; psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]
    unit_cell = periodic_system(lattice, atoms, positions)

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE, make the supercell
    # and then convert back ...
    supercell_ase = convert_ase(unit_cell) * pytuple((repeat, 1, 1))
    supercell     = pyconvert(AbstractSystem, supercell_ase)

    # Unfortunately right now the conversion to ASE drops the pseudopotential information,
    # so we need to reattach it:
    supercell = attach_psp(supercell; Al="hgh/lda/al-q3")

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(supercell; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

    CondaPkg Found dependencies: /home/runner/.julia/packages/ASEconvert/CNQ1A/CondaPkg.toml
    CondaPkg Found dependencies: /home/runner/.julia/packages/PythonCall/wXfah/CondaPkg.toml
    CondaPkg Dependencies already up to date


As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
convert_ase(periodic_system(setup.model)).write("al_supercell.png")

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123


Python: None

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298219324047                   -0.85    5.0         
  2   -8.300174494067       -2.71       -1.26    1.0   88.5ms
  3   -8.300427692539       -3.60       -1.89    3.2    117ms
  4   -8.300461126037       -4.48       -2.76    1.9    105ms
  5   -8.300463889514       -5.56       -3.04    3.1    155ms
  6   -8.300464277057       -6.41       -3.25    4.0    155ms
  7   -8.300464461906       -6.73       -3.40    8.8    201ms
  8   -8.300464560466       -7.01       -3.54    2.0    104ms
  9   -8.300464620985       -7.22       -3.74    1.2   87.5ms
 10   -8.300464635075       -7.85       -3.90    1.5    104ms
 11   -8.300464643340       -8.08       -4.22    3.0    157ms


In [4]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -16.66872894433                   -0.70    6.6         
  2   -16.67857595549       -2.01       -1.14    1.5    232ms
  3   -16.67921041794       -3.20       -1.86    3.6    253ms
  4   -16.67927886634       -4.16       -2.72    2.2    225ms
  5   -16.67928576020       -5.16       -3.09    4.8    318ms
  6   -16.67928619759       -6.36       -3.50    2.4    224ms
  7   -16.67928621772       -7.70       -3.94    2.8    229ms
  8   -16.67928622149       -8.42       -4.53    2.1    252ms


In [5]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32686422266                   -0.56    7.1         
  2   -33.33440413450       -2.12       -1.00    1.2    692ms
  3   -33.33599901170       -2.80       -1.72    4.5    929ms
  4   -33.33617255630       -3.76       -2.63    4.4    858ms
  5   -33.33691897120       -3.13       -2.61    8.1    1.25s
  6   -33.33693556148       -4.78       -2.88    3.8    777ms
  7   -33.33694361513       -5.09       -3.71    2.9    811ms
  8   -33.33694371200       -7.01       -3.84    4.9    911ms
  9   -33.33694377922       -7.17       -4.44    1.4    660ms


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298406294900                   -0.85    5.0         
  2   -8.300231731259       -2.74       -1.58    1.0   59.7ms
  3   -8.300224497137   +   -5.14       -2.07    2.2   77.2ms
  4   -8.300364002379       -3.86       -2.25    3.0   96.2ms
  5   -8.300463882517       -4.00       -3.51    1.1   64.9ms
┌ Warning: Eigensolver not converged
│   n_iter =
│    8-element Vector{Int64}:
│     100
│       3
│       3
│       3
│       3
│       3
│       5
│       2
└ @ DFTK ~/work/DFTK.jl/DFTK.jl/src/scf/self_consistent_field.jl:35
  6   -8.300464478954       -6.22       -3.68   15.2    556ms
  7   -8.300464629115       -6.82       -4.12    2.8    107ms


In [7]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32605332341                   -0.56    6.6         
  2   -33.30324400494   +   -1.64       -1.26    1.4    600ms
  3   -2.682915570907   +    1.49       -0.32    7.0    1.52s
  4   -33.31290875966        1.49       -1.57    5.8    1.13s
  5   -33.23768072526   +   -1.12       -1.37    3.8    862ms
  6   -30.42207956673   +    0.45       -0.83    4.0    945ms
  7   -33.30843198861        0.46       -1.78    5.1    1.00s
  8   -33.33497698776       -1.58       -2.27    2.4    692ms
  9   -33.33553692218       -3.25       -2.25    2.9    898ms
 10   -33.33549118689   +   -4.34       -2.47    1.8    701ms
 11   -33.33679873781       -2.88       -2.86    2.0    640ms
 12   -33.33682810490       -4.53      

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.